<a href="https://colab.research.google.com/github/Niveditathakur/Regression/blob/master/regression_MLP_1_D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm

import seaborn as sns

from sklearn import model_selection, preprocessing, decomposition
from sklearn.utils import shuffle
from matplotlib.patches import Ellipse

from sklearn.preprocessing import LabelEncoder,  StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, log_loss, classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from keras.utils import np_utils
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

seed = 7
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))


from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings("ignore")


In [0]:
# load dataset
from sklearn.datasets import load_boston
boston_data = load_boston()

print('Boston Data Input Shape:', boston_data.data.shape)
print('Boston Data Target Shape:', boston_data.target.shape)
print('Boston Data Feature Names:', boston_data.feature_names)

# Following line will print the detailed description of the data
#print('Feature Description:', boston_dataset.DESCR)

In [0]:
# convert the input boston data feature into a pandas dataframe with columns as features
#print( boston.head(3) )
boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
print( boston.head(3) )

#plot histogram of input featues to see if feature scaling is required
boston.hist()
plt.show()

#we can check if there is a null value - this dataset has none
#boston.isnull().sum()

In [0]:
boston['Price'] = boston_data.target
#derive data and labels
data   = boston.drop('Price', axis = 1)  
labels = boston['Price']

In [0]:
# Use a simple MLP for the Regression task
# epochs and batch size 
num_folds = 3 
#err_range = 5
num_epochs = 100
batch_size = 5 

def base_model():
    # create model
    model = Sequential()

    model.add(Dense(16, input_dim=data.shape[1], init = 'normal', activation='relu'))
    model.add(Dropout(0.2)) # dropout
    model.add(Dense(32, init = 'normal', activation='relu'))
    model.add(Dropout(0.2)) 
    #we will not use softmax here as not a classification task
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

reg         =  KerasRegressor(build_fn=base_model, epochs = num_epochs, batch_size = batch_size, verbose=0)
kfold       =  KFold(n_splits=num_folds, shuffle=True, random_state=seed)

results     = cross_val_score(reg, data, labels, cv=kfold)
print("Results RMSE with confidence interval: %.2f (%.2f)" % (np.sqrt(np.abs(results.mean())), np.sqrt(np.abs(results.std()))))

Results RMSE with confidence interval: 5.95 (1.85)


In [0]:
# Employ 1-D CNN for regression 
def base_model():
    # create model
    model = Sequential()
    model.add(Conv1D(16, input_shape=(data.shape[1],1), kernel_size=5, strides=2, padding='valid'))
    model.add(BatchNormalization()), # batch normalization
    model.add(MaxPooling1D(3)), 
    model.add(Activation('relu')),
    model.add(Dropout(0.1)),
    model.add(Flatten()),
    model.add(Dense(16, init = 'normal', activation='relu'))
    model.add(Dropout(0.1)) 
    #we will not use softmax here as not a classification task
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model
  
reg         =  KerasRegressor(build_fn=base_model, epochs = num_epochs, batch_size = batch_size, verbose=0)
kfold       =  KFold(n_splits=num_folds, shuffle=True, random_state=seed)

#reshape input data for the input of CNN

data_cnn    = data.values.reshape(data.shape[0],  data.shape[1],  1) 

results     = cross_val_score(reg, data_cnn, labels, cv=kfold)

print("Results RMSE with confidence interval: %.2f (%.2f)" % (np.sqrt(np.abs(results.mean())), np.sqrt(np.abs(results.std()))))

#1D CNN is performing worst in this case as compared to MLP

Results RMSE with confidence interval: 7.81 (2.64)
